# Demonstrație Algoritmul Simon

Algoritmul Simon este unul dintre algoritmii cuantici fundamentali care demonstrează o separație exponențială între calculul clasic și cel cuantic pentru problema găsirii periodicității unei funcții.

## Problema Simon

Dată o funcție f: {0,1}ⁿ → {0,1}ⁿ cu proprietatea că există un șir secret s ∈ {0,1}ⁿ astfel încât:

f(x) = f(y) dacă și numai dacă y = x sau y = x ⊕ s

**Obiectiv**: Găsirea șirului secret s

**Clasic**: O(2^(n/2)) evaluări (algoritm probabilistic)
**Cuantic**: O(n) evaluări!

Aceasta este o problemă care a inspirat algoritmi mai mari precum algoritmul Shor.

In [ ]:
# Importuri necesare
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, circuit_drawer
import matplotlib.pyplot as plt
from itertools import product
import random

print("Librării importate cu succes!")

## Fundamentele Matematice

### Funcția Simon
Pentru un șir secret s ≠ 0ⁿ, funcția f satisface:
- f(x) = f(x ⊕ s) pentru orice x
- f(x) ≠ f(y) dacă y ≠ x și y ≠ x ⊕ s

### Strategia Clasică
Algoritmul clasic optim (Birthday Paradox):
1. Evaluează f pentru ~√(2^n) = 2^(n/2) valori aleatorii
2. Caută coliziuni f(x) = f(y)
3. Calculează s = x ⊕ y

### Strategia Cuantică
1. Creează superpoziție: ∑|x⟩|0⟩
2. Aplică oracolul: ∑|x⟩|f(x)⟩
3. Măsoară registrul de output pentru a "colecta" perechi
4. Aplică Hadamard și măsoară pentru a obține relații liniare
5. Rezolvă sistemul liniar pentru a găsi s

In [ ]:
def create_simon_oracle(secret_string):
    """Creează un oracol pentru problema Simon cu șirul secret dat"""
    n = len(secret_string)
    
    # Pentru simplitate, vom crea o funcție care mapează:
    # x -> x dacă x < x⊕s
    # x -> x⊕s dacă x >= x⊕s
    # Astfel f(x) = f(x⊕s)
    
    oracle = QuantumCircuit(2 * n, name=f'Simon Oracle s={secret_string}')
    
    # Implementăm o funcție simplă care respectă proprietatea Simon
    # Pentru fiecare bit din secret, dacă este 1, aplicăm o operație specifică
    
    # Copiem input-ul în output (funcție identitate modificată)
    for i in range(n):
        oracle.cx(i, n + i)
    
    # Aplicăm modificări bazate pe șirul secret
    # Aceasta este o implementare simplificată
    for i, bit in enumerate(secret_string):
        if bit == '1':
            # Adăugăm niște complexitate pentru a face funcția non-trivială
            oracle.cx(i, n + (i + 1) % n)
    
    return oracle

def simon_algorithm_single_query(secret_string):
    """O singură interogare a algoritmului Simon"""
    n = len(secret_string)
    
    # Circuitul are 2n qubiti (n pentru input, n pentru output) și n biți clasici
    qc = QuantumCircuit(2 * n, n)
    
    # Pasul 1: Creăm superpoziție pe primii n qubiti
    for i in range(n):
        qc.h(i)
    
    # Pasul 2: Aplicăm oracolul
    oracle = create_simon_oracle(secret_string)
    qc = qc.compose(oracle)
    
    # Pasul 3: Aplicăm Hadamard pe primii n qubiti
    for i in range(n):
        qc.h(i)
    
    # Pasul 4: Măsurăm primii n qubiti
    for i in range(n):
        qc.measure(i, i)
    
    return qc

# Testăm cu diferite șiruri secrete
test_secrets = ['01', '10', '011', '101', '110']

for secret in test_secrets:
    if '1' not in secret:  # Sărim cazul trivial s = 000...
        continue
    print(f"\nȘirul secret: {secret}")
    oracle = create_simon_oracle(secret)
    print(f"Oracolul pentru s={secret}:")
    print(oracle.draw())

In [ ]:
def binary_gaussian_elimination(matrix):
    """Eliminare Gaussiană în aritmetica binară (mod 2)"""
    if not matrix:
        return []
    
    rows = len(matrix)
    cols = len(matrix[0])
    
    # Copiem matricea pentru a nu modifica originalul
    mat = [row[:] for row in matrix]
    
    # Forward elimination
    rank = 0
    for col in range(cols - 1):  # Ultimul coloană este pentru rezultat (0)
        # Găsim primul rând cu 1 în coloana curentă
        pivot_row = None
        for row in range(rank, rows):
            if mat[row][col] == 1:
                pivot_row = row
                break
        
        if pivot_row is None:
            continue  # Coloana este deja zero
        
        # Schimbăm rândurile dacă e necesar
        if pivot_row != rank:
            mat[rank], mat[pivot_row] = mat[pivot_row], mat[rank]
        
        # Eliminăm celelalte 1-uri din coloană
        for row in range(rows):
            if row != rank and mat[row][col] == 1:
                for c in range(cols):
                    mat[row][c] ^= mat[rank][c]
        
        rank += 1
    
    return mat

def solve_simon_system(measurements, n):
    """Rezolvă sistemul liniar pentru a găsi șirul secret Simon"""
    # Filtrăm măsurătorile non-zero
    non_zero_measurements = [m for m in measurements if m != '0' * n]
    
    if len(non_zero_measurements) < n - 1:
        return None  # Nu avem suficiente ecuații
    
    # Construim matricea sistemului y·s = 0 (mod 2)
    matrix = []
    for measurement in non_zero_measurements[:n-1]:  # Avem nevoie de n-1 ecuații independente
        row = [int(bit) for bit in measurement] + [0]  # Adăugăm 0 pentru partea dreaptă
        matrix.append(row)
    
    # Rezolvăm prin eliminare Gaussiană
    reduced_matrix = binary_gaussian_elimination(matrix)
    
    # Găsim soluția în spațiul null
    # Pentru simplitate, încercăm toate valorile posibile pentru ultima variabilă
    for last_bit in [0, 1]:
        solution = [0] * n
        solution[-1] = last_bit
        
        # Back substitution
        for i in range(min(len(reduced_matrix), n-1) - 1, -1, -1):
            if i < len(reduced_matrix) and any(reduced_matrix[i][:n]):
                # Găsim primul bit non-zero
                pivot_col = next(j for j in range(n) if reduced_matrix[i][j] == 1)
                
                # Calculăm valoarea
                val = 0
                for j in range(pivot_col + 1, n):
                    val ^= reduced_matrix[i][j] * solution[j]
                
                solution[pivot_col] = val
        
        # Verificăm dacă soluția este non-trivială
        if any(solution):
            return ''.join(map(str, solution))
    
    return None

# Test pentru funcțiile de algebră liniară
print("Test eliminare Gaussiană binară:")
test_matrix = [
    [1, 0, 1, 0],
    [0, 1, 1, 0],
    [1, 1, 0, 0]
]
result = binary_gaussian_elimination(test_matrix)
print(f"Matricea originală: {test_matrix}")
print(f"Matricea redusă: {result}")

In [ ]:
# Implementarea completă a algoritmului Simon
def simon_algorithm_complete(secret_string, max_iterations=10):
    """Algoritmul Simon complet cu multiple interogări"""
    n = len(secret_string)
    measurements = []
    
    simulator = AerSimulator()
    shots_per_iteration = 100
    
    print(f"\nRulează algoritmul Simon pentru s = {secret_string}")
    print(f"Avem nevoie de ~{n-1} măsurători independente")
    
    for iteration in range(max_iterations):
        # Creăm și rulăm circuitul
        qc = simon_algorithm_single_query(secret_string)
        transpiled = transpile(qc, simulator)
        job = simulator.run(transpiled, shots=shots_per_iteration)
        result = job.result()
        counts = result.get_counts()
        
        # Colectăm măsurătorile
        for measurement, count in counts.items():
            # Adăugăm măsurătorile cu probabilitate proporțională cu numărul lor
            for _ in range(min(count, 5)):  # Limităm pentru eficiență
                measurements.append(measurement)
        
        # Încercăm să rezolvăm sistemul după fiecare iterație
        if len(measurements) >= n - 1:
            unique_measurements = list(set(measurements))
            solution = solve_simon_system(unique_measurements, n)
            
            if solution and solution != '0' * n:
                print(f"\nSoluție găsită după {iteration + 1} iterații!")
                print(f"Măsurători colectate: {unique_measurements[:10]}...")  # Primele 10
                print(f"Șirul secret detectat: {solution}")
                return solution, iteration + 1, unique_measurements
    
    print(f"\nNu s-a găsit soluție după {max_iterations} iterații")
    return None, max_iterations, list(set(measurements))

# Testăm algoritmul pentru diferite șiruri secrete
test_secrets_filtered = [s for s in test_secrets if '1' in s]  # Excludem cazul trivial

results = []
for secret in test_secrets_filtered:
    solution, iterations, measurements = simon_algorithm_complete(secret)
    results.append((secret, solution, iterations, measurements))
    
    print(f"\n{'='*50}")
    if solution == secret:
        print(f"✅ SUCCESS pentru s = {secret}!")
    elif solution:
        print(f"⚠️  Soluție parțială pentru s = {secret}: găsit {solution}")
    else:
        print(f"❌ FAILED pentru s = {secret}")
    print(f"Iterații necesare: {iterations}")
    print(f"{'='*50}")

In [ ]:
# Vizualizăm rezultatele algoritmului Simon
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for i, (secret, solution, iterations, measurements) in enumerate(results[:4]):
    ax = axes[i]
    
    # Analizăm distribuția măsurătorilor
    measurement_counts = {}
    for m in measurements:
        measurement_counts[m] = measurement_counts.get(m, 0) + 1
    
    # Sortăm și luăm primele 8 rezultate
    sorted_measurements = dict(sorted(measurement_counts.items(), 
                                    key=lambda x: x[1], reverse=True)[:8])
    
    labels = list(sorted_measurements.keys())
    values = list(sorted_measurements.values())
    
    # Colorare: verde pentru măsurătorile care respectă s·y = 0
    colors = []
    for label in labels:
        if solution:
            # Verificăm dacă label · solution = 0 (mod 2)
            dot_product = sum(int(label[j]) * int(solution[j]) for j in range(len(label))) % 2
            colors.append('lightgreen' if dot_product == 0 else 'lightcoral')
        else:
            colors.append('lightblue')
    
    bars = ax.bar(range(len(labels)), values, color=colors)
    ax.set_title(f'Secret: {secret}\nSoluție: {solution or "None"}\nIterații: {iterations}', 
                fontsize=10, weight='bold')
    ax.set_ylabel('Frecvența măsurătorii')
    ax.set_xlabel('Măsurători')
    ax.set_xticks(range(len(labels)))
    ax.set_xticklabels(labels, rotation=45, ha='right', fontsize=8)
    
    # Adăugăm valorile pe bare
    for bar, value in zip(bars, values):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{value}',
                ha='center', va='bottom', fontsize=8)
    
    # Indicator de succes
    success = solution == secret
    status_color = 'green' if success else 'orange' if solution else 'red'
    status_text = '✅' if success else '⚠️' if solution else '❌'
    
    ax.text(0.02, 0.98, status_text, transform=ax.transAxes,
            fontsize=16, color=status_color,
            va='top', ha='left',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

# Legendă pentru culori
print("\nLegenda culorilor:")
print("🟢 Verde: Măsurătorile care satisfac y·s = 0 (mod 2)")
print("🔴 Roșu: Măsurătorile care nu satisfac condiția")
print("🔵 Albastru: Măsurătorile când nu s-a găsit soluție")

In [ ]:
def classical_simon_algorithm(oracle_function, n, max_queries=None):
    """Simularea algoritmului clasic pentru problema Simon (Birthday Paradox)"""
    if max_queries is None:
        max_queries = 2**(n//2 + 2)  # ~√(2^n) cu un factor de siguranță
    
    evaluations = {}
    queries = 0
    
    # Generăm input-uri aleatorii și căutăm coliziuni
    for _ in range(max_queries):
        # Generăm un input aleator
        x = ''.join(random.choice('01') for _ in range(n))
        queries += 1
        
        # Evaluăm funcția
        f_x = oracle_function(x)
        
        # Căutăm coliziuni
        if f_x in evaluations:
            # Găsit coliziune!
            y = evaluations[f_x]
            if y != x:  # Evităm coliziunea trivială
                # s = x ⊕ y
                secret = ''.join(str(int(x[i]) ^ int(y[i])) for i in range(n))
                return secret, queries
        else:
            evaluations[f_x] = x
    
    return None, queries

def create_simon_oracle_function(secret_string):
    """Creează o funcție oracol pentru problema Simon"""
    n = len(secret_string)
    
    def oracle(x):
        # Pentru simplitate, implementăm f(x) = x dacă x < x⊕s, altfel f(x) = x⊕s
        x_int = int(x, 2)
        s_int = int(secret_string, 2)
        x_xor_s = x_int ^ s_int
        
        if x_int <= x_xor_s:
            result = x_int
        else:
            result = x_xor_s
        
        return format(result, f'0{n}b')
    
    return oracle

# Comparăm algoritmul clasic cu cel cuantic
print("COMPARAȚIA CLASIC vs CUANTIC pentru Algoritmul Simon\n")
print(f"{'Secret':<8} {'Clasic':<10} {'Queries':<8} {'Cuantic':<10} {'Iter.':<5} {'Speedup':<10}")
print("-" * 70)

for secret, solution, iterations, _ in results:
    # Algoritmul clasic
    oracle_func = create_simon_oracle_function(secret)
    classical_result, classical_queries = classical_simon_algorithm(oracle_func, len(secret))
    
    # Calculăm speedup-ul
    if classical_queries and iterations:
        speedup = f"{classical_queries/iterations:.1f}x"
    else:
        speedup = "N/A"
    
    classical_success = "✅" if classical_result == secret else "❌"
    quantum_success = "✅" if solution == secret else "❌"
    
    print(f"{secret:<8} {classical_success:<10} {classical_queries:<8} {quantum_success:<10} {iterations:<5} {speedup:<10}")

print("\n" + "="*70)
print("AVANTAJUL CUANTIC:")
print(f"- Clasic: O(2^(n/2)) queries (Birthday Paradox)")
print(f"- Cuantic: O(n) queries")
print(f"- Pentru n mari: Speedup exponențial!")
print("- Algoritmul Simon a inspirat algoritmul Shor pentru factorizare")
print("="*70)

In [ ]:
# Demonstrație detaliată pentru un exemplu simplu
simple_secret = '10'
n = len(simple_secret)

print(f"DEMONSTRAȚIE PAS CU PAS pentru secret s = {simple_secret}")
print("="*60)

# Explicăm funcția
print(f"\nFuncția Simon pentru s = {simple_secret}:")
oracle_func = create_simon_oracle_function(simple_secret)

print(f"\n{'x':<4} {'f(x)':<6} {'x⊕s':<6} {'f(x⊕s)':<8} {'f(x)=f(x⊕s)?':<15}")
print("-" * 45)

for x_int in range(2**n):
    x = format(x_int, f'0{n}b')
    f_x = oracle_func(x)
    
    x_xor_s = x_int ^ int(simple_secret, 2)
    x_xor_s_str = format(x_xor_s, f'0{n}b')
    f_x_xor_s = oracle_func(x_xor_s_str)
    
    matches = "✅" if f_x == f_x_xor_s else "❌"
    
    print(f"{x:<4} {f_x:<6} {x_xor_s_str:<6} {f_x_xor_s:<8} {matches:<15}")

print(f"\nProprietatea Simon: f(x) = f(x ⊕ {simple_secret}) pentru orice x")

# Simulăm o iterație a algoritmului cuantic
print(f"\n\nIterația algoritmului cuantic:")
qc_demo = simon_algorithm_single_query(simple_secret)
print(f"Circuitul pentru s = {simple_secret}:")
print(qc_demo.draw())

# Rulăm simularea
simulator = AerSimulator()
transpiled_demo = transpile(qc_demo, simulator)
job_demo = simulator.run(transpiled_demo, shots=1000)
result_demo = job_demo.result()
counts_demo = result_demo.get_counts()

print(f"\nRezultatele măsurătorii:")
for measurement, count in sorted(counts_demo.items(), key=lambda x: x[1], reverse=True):
    percentage = count / 1000 * 100
    # Verificăm dacă measurement · secret = 0 (mod 2)
    dot_product = sum(int(measurement[i]) * int(simple_secret[i]) for i in range(n)) % 2
    valid = "✅" if dot_product == 0 else "❌"
    print(f"{measurement}: {count:4d} ({percentage:5.1f}%) - y·s = {dot_product} {valid}")

print(f"\nObservație: Măsurătorile y cu y·s = 0 (mod 2) au probabilitate mai mare!")
print(f"Din aceste măsurători putem reconstitui s prin algebra liniară.")

## Aplicații și Impactul Istoric

### Algoritmul Shor
Problema Simon a inspirat direct algoritmul Shor pentru factorizarea numerelor întregi:
- **Problema comună**: Găsirea periodicității unei funcții
- **Adaptare**: Shor folosește problema periodicității pentru a găsi ordinul unui element
- **Impact**: Amenință siguranța criptografiei RSA

### Alte Aplicații

1. **Criptanaliză**: Atacuri asupra funcțiilor hash și cifrurilor simetrice
2. **Teoria complexității**: Demonstrarea separării BQP vs BPP
3. **Algoritmi cuantici**: Fundament pentru algoritmi de căutare structurată

### Complexitate Teoretică

**Clasic**: 
- Cel mai bun algoritm cunoscut: O(2^(n/2)) queries
- Folosește Birthday Paradox pentru găsirea coliziunilor

**Cuantic**:
- Algoritmul Simon: O(n) queries
- Speedup exponențial pentru n mare
- Prima demonstrare de separație exponențială pentru o problemă relativ naturală

In [ ]:
# Analiză teoretică a complexității
import math

def complexity_analysis():
    """Analizează complexitatea pentru diferite dimensiuni de problemă"""
    sizes = list(range(2, 11))
    
    print("ANALIZA COMPLEXITĂȚII ALGORITMULUI SIMON\n")
    print(f"{'n':<3} {'Clasic (2^(n/2))':<15} {'Cuantic (n)':<12} {'Speedup':<15} {'Ratio':<10}")
    print("-" * 70)
    
    for n in sizes:
        classical_complexity = 2**(n/2)
        quantum_complexity = n
        speedup = classical_complexity / quantum_complexity
        ratio = f"2^{n/2:.1f}/n" if n > 2 else "1"
        
        print(f"{n:<3} {classical_complexity:<15.1f} {quantum_complexity:<12} {speedup:<15.1f} {ratio:<10}")
    
    print("\n" + "="*70)
    print("OBSERVAȚII:")
    print("- Speedup-ul crește exponențial cu n")
    print("- Pentru n=10: speedup de ~5.7x")
    print("- Pentru n=20: speedup de ~51.2x")
    print("- Pentru n=100: speedup de ~3.2 × 10^13x")
    print("="*70)

complexity_analysis()

# Vizualizare grafică a complexității
sizes = list(range(2, 16))
classical_comp = [2**(n/2) for n in sizes]
quantum_comp = sizes

plt.figure(figsize=(12, 8))

plt.subplot(2, 1, 1)
plt.semilogy(sizes, classical_comp, 'r-o', label='Clasic: O(2^(n/2))', linewidth=2, markersize=6)
plt.plot(sizes, quantum_comp, 'b-s', label='Cuantic: O(n)', linewidth=2, markersize=6)
plt.xlabel('Dimensiunea problemei (n)')
plt.ylabel('Numărul de queries (log scale)')
plt.title('Complexitatea Algoritmului Simon: Clasic vs Cuantic')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 1, 2)
speedups = [c/q for c, q in zip(classical_comp, quantum_comp)]
plt.semilogy(sizes, speedups, 'g-^', label='Speedup = Clasic/Cuantic', linewidth=2, markersize=6)
plt.xlabel('Dimensiunea problemei (n)')
plt.ylabel('Speedup (log scale)')
plt.title('Speedup-ul Cuantic pentru Algoritmul Simon')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Concluzie

Algoritmul Simon reprezintă o piatră de hotar în istoria calculului cuantic:

### Importanța Istorică

1. **Prima separație exponențială**: Pentru o problemă relativ naturală și concretă
2. **Inspirația pentru Shor**: A deschis calea către factorizarea cuantică
3. **Impact asupra criptografiei**: A demonstrat vulnerabilitatea potențială a sistemelor criptografice

### Principii Demonstrații

1. **Interferența cuantică constructivă**: Pentru extragerea informației despre structura funcției
2. **Periodicitatea cuantică**: Exploatarea simetriilor ascunse
3. **Algebra liniară cuantică**: Folosirea măsurătorilor pentru a construi sisteme de ecuații

### Avantaje Practice

- **Speedup exponențial**: De la O(2^(n/2)) la O(n)
- **Deterministic**: În absența zgomotului, garantează găsirea soluției
- **Generalizabil**: Principiile se aplică la probleme similare de periodicitate

### Limitări și Provocări

1. **Specificitatea problemei**: Funcționează doar pentru funcții cu proprietatea Simon
2. **Implementarea practică**: Necesită coherență cuantică pe durate lungi
3. **Corectarea erorilor**: Zgomotul poate afecta semnificativ rezultatele

### Lecții pentru Viitor

Algoritmul Simon ne învață că:
- Structura problemei poate fi exploatată cuantic în moduri neobișnuite
- Interferența cuantică poate amplifica semnale slabe
- Problemele de periodicitate sunt natural potrivite pentru calculul cuantic

Această înțelegere continuă să inspire noi algoritmi cuantici și să ghideze dezvoltarea calculatoarelor cuantice către aplicații practice revolutionare.